# 102 - Explore Retrospective
As an exmaple of a retrospective evaluation, we cached simulated and observed flows of 49 gages in HUC4 1802 in California, for both NWM v2.0 and NWM v2.1.  This gives us 2 simulated timeseries to compare to observed, to assess and start to think about standardizing the way we answer the question, "which is better".

There is significant work in this area to identify different ways to compare metrics between simulations.

In [ ]:
# Import the required packages.
import sys
from pathlib import Path
import duckdb
import pandas as pd
import geopandas as gpd

import holoviews as hv
import geoviews as gv
import teehr.queries.duckdb as tqd
import teehr.queries.pandas as tqp
import hvplot.pandas
import cartopy.crs as ccrs
from holoviews import opts

In [ ]:
# Set some notebook variables to point to the relevant study files.
USGS_TIMESERIES_FILE = Path(Path().home(), "cache/huc1802_retro/timeseries/usgs.parquet")
NWM21_TIMESERIES_FILE = Path(Path().home(), "cache/huc1802_retro/timeseries/nwm21_retro.parquet")
NWM20_TIMESERIES_FILE = Path(Path().home(), "cache/huc1802_retro/timeseries/nwm20_retro.parquet")

USGS_NWM22_CROSSWALK = Path(Path().home(), "shared/rti-eval/post-event-example/geo/usgs_nwm22_crosswalk.parquet")
USGS_HUC12_CROSSWALK = Path(Path().home(), "shared/rti-eval/post-event-example/geo/usgs_huc12_crosswalk.parquet")
USGS_GEOMETRY = Path(Path().home(), "shared/rti-eval/post-event-example/geo/usgs_geometry.parquet")

In [ ]:
# ?tqd.get_metrics

In [ ]:
filters = [
    {
        "column": "value_time",
        "operator": ">=",
        "value": "1993-01-01"
    },
    {
        "column": "value_time",
        "operator": "<",
        "value": "2019-01-01"
    },
    {
        "column": "primary_value",
        "operator": ">",
        "value": 0
    },
]
variable_name = "bias"

In [ ]:
%%time
nwm21_metrics = tqd.get_metrics(
        primary_filepath=USGS_TIMESERIES_FILE,
        secondary_filepath=NWM21_TIMESERIES_FILE,
        crosswalk_filepath=USGS_NWM22_CROSSWALK,
        geometry_filepath=USGS_GEOMETRY,
        group_by=["primary_location_id"],
        order_by=["primary_location_id"],
        include_metrics=[variable_name],
        return_query=False,
        include_geometry=True,
        filters=filters
)

In [ ]:
nwm21_metrics.head()

In [ ]:
%%time
nwm20_metrics = tqd.get_metrics(
        primary_filepath=USGS_TIMESERIES_FILE,
        secondary_filepath=NWM20_TIMESERIES_FILE,
        crosswalk_filepath=USGS_NWM22_CROSSWALK,
        geometry_filepath=USGS_GEOMETRY,
        group_by=["primary_location_id"],
        order_by=["primary_location_id"],
        include_metrics=[variable_name],
        return_query=False,
        include_geometry=True,
        filters=filters
)

In [ ]:
nwm20_metrics.head()

In [ ]:
nwm_compare = nwm20_metrics.merge(
    nwm21_metrics,
    on="primary_location_id",
    suffixes=('_nwm20', '_nwm21')
)

In [ ]:
nwm_compare[f"abs_{variable_name}_nwm21"] = nwm_compare[f"{variable_name}_nwm21"].abs()
nwm_compare[f"abs_{variable_name}_nwm20"] = nwm_compare[f"{variable_name}_nwm20"].abs()
nwm_compare[f"abs_{variable_name}_delta"] = nwm_compare[f"abs_{variable_name}_nwm21"] - nwm_compare[f"abs_{variable_name}_nwm20"]
nwm_compare.sort_values(by=f"abs_{variable_name}_delta", inplace=True)
nwm_compare[[
    "primary_location_id",
    f"{variable_name}_nwm20", 
    f"{variable_name}_nwm21", 
    f"abs_{variable_name}_nwm21", 
    f"abs_{variable_name}_nwm20",
    f"abs_{variable_name}_delta"
]]

In [ ]:
compare_gdf = gpd.GeoDataFrame(nwm_compare, geometry="geometry_nwm20").to_crs("EPSG:3857")

In [ ]:
tiles = gv.tile_sources.OSM
usgs = compare_gdf.hvplot(c=f"abs_{variable_name}_delta", hover_cols=["primary_location_id"], crs=ccrs.GOOGLE_MERCATOR, cmap="bgy")
(usgs * tiles).opts(width=600, height=600, show_legend=False)

In [ ]:
%%time
filters=[
    {
        "column": "primary_location_id",
        "operator": "in",
        "value": ["usgs-11345500", "usgs-11447905"]
    }
]
nwm21_timeseries = tqd.get_joined_timeseries(
        primary_filepath=USGS_TIMESERIES_FILE,
        secondary_filepath=NWM21_TIMESERIES_FILE,
        crosswalk_filepath=USGS_NWM22_CROSSWALK,
        order_by=["primary_location_id", "value_time"],
        return_query=False,
        filters=filters
)
nwm20_timeseries = tqd.get_joined_timeseries(
        primary_filepath=USGS_TIMESERIES_FILE,
        secondary_filepath=NWM20_TIMESERIES_FILE,
        crosswalk_filepath=USGS_NWM22_CROSSWALK,
        order_by=["primary_location_id", "value_time"],
        return_query=False,
        filters=filters
)
merged_timeseries = nwm20_timeseries.merge(
    nwm21_timeseries,
    on=["primary_location_id", "value_time"],
    suffixes=('_nwm20', '_nwm21')
)

In [ ]:
merged_timeseries = merged_timeseries[["primary_location_id", "value_time", "primary_value_nwm20", "secondary_value_nwm20", "secondary_value_nwm21"]]

In [ ]:
merged_timeseries.loc[
    merged_timeseries["primary_location_id"]=="usgs-11345500"
].hvplot(x="value_time", y=["primary_value_nwm20", "secondary_value_nwm20", "secondary_value_nwm21"]).opts(width=1000)

In [ ]:
merged_timeseries.loc[
    merged_timeseries["primary_location_id"]=="usgs-11447905"
].hvplot(x="value_time", y=["primary_value_nwm20", "secondary_value_nwm20", "secondary_value_nwm21"]).opts(width=1000)